In [ ]:
import pandas as pd
import  random
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
from datetime import datetime, date
from faker import Faker

fake = Faker()

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP" #"HIST_ATUALIZACOES"
driver = "ODBC Driver 17 for SQL Server"

conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring, pool_size=10, max_overflow=30)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            print("Conexão com o banco de dados estabelecida com sucesso")
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print("Query executada com sucesso")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
if testar_conexao_sqlalchemy(engine): 
    print("A conexão foi bem-sucedida")
else:
    print("Falha na conexão.")


#DATA DA NOTA
data_nota_ini = fake.date_time_between(start_date=datetime(2024, 1, 1), end_date=datetime(2024, 1, 20))
hora_aleatoria = random.randint(7, 21)  #Gera uma hora entre 8 e 22
minuto_aleatorio = random.randint(0, 59)  #Gera minutos aleatórios
segundo_aleatorio = random.randint(0, 59)  #Gera segundos aleatórios
data_nota = data_nota_ini.replace(hour=hora_aleatoria, minute=minuto_aleatorio, second=segundo_aleatorio) #Substituir o horário da data_nota pelo horário gerado

#CRIAR O NUMERO DA NOTA FISCAL

#try:
#    with engine.connect() as conn:
#            
#            qtde_nf_no_bancodados = conn.execute(text("SELECT COUNT(NUMERONF) FROM NOTAFISCAL")).scalar()
#            
#            print(f"Numero da ultima nota fiscal do banco de dados: {qtde_nf_no_bancodados}")            
#except Exception as e:
#    print(f"Erro ao executar: {e}")
#
num_notas = 3

lista_notas_fiscais = []
lista_ids_nfs = []

df_tabela_produto = pd.read_sql("SELECT IDPRODUTO,PRECO_CUSTO,PRECO_UNIT FROM PRODUTO",engine)
id_produto = df_tabela_produto['IDPRODUTO'].count()
lista_clientes = (pd.read_sql("SELECT IDCLIENTE FROM CLIENTE", engine)['IDCLIENTE']).to_list()
lista_lojas = (pd.read_sql("SELECT IDLOJA FROM LOJA", engine)['IDLOJA']).to_list()
lista_forma_pagamento = (pd.read_sql("SELECT IDFORMAPGMT FROM FORMA_PAGAMENTO", engine)['IDFORMAPGMT']).to_list()
lista_id_cfop = (pd.read_sql("SELECT IDCODCFOP FROM CFOP", engine)['IDCODCFOP']).to_list()
lista_chave_acesso_db = (pd.read_sql("SELECT CHAVE_ACESSO FROM NOTAFISCAL", engine)['CHAVE_ACESSO']).to_list()

display(df_tabela_produto)

qtde_nf_no_bancodados = engine.connect().execute(text("SELECT COUNT(IDNOTAFISCAL) FROM NOTAFISCAL")).scalar()

for _ in range(num_notas):
    id_notafiscal = qtde_nf_no_bancodados + 1
    qtde_itens_nf = int(random.randint(1,10))

    #data da nota
    data_nota_ini = fake.date_time_between(start_date=datetime(2024, 1, 1), end_date=datetime(2024, 1, 20))
    hora_aleatoria = random.randint(7, 21)  #Gera uma hora entre 8 e 22
    minuto_aleatorio = random.randint(0, 59)  #Gera minutos aleatórios
    segundo_aleatorio = random.randint(0, 59)  #Gera segundos aleatórios
    data_nota = data_nota_ini.replace(hour=hora_aleatoria, minute=minuto_aleatorio, second=segundo_aleatorio)
    for _ in range(qtde_itens_nf):
        id_produto = random.randint(1,(df_tabela_produto['IDPRODUTO'].count()))
        quantidade_produtos = random.randint(1,10)
        preco_unit = round(df_tabela_produto.loc[df_tabela_produto["IDPRODUTO"] == id_produto, "PRECO_UNIT"].values[0],2)
        total_item = round((quantidade_produtos*preco_unit), 2)
        data_criacao = datetime.today().date()
        data_atlz = None
        versao = 1
        
        
        lista_notas_fiscais.append((id_notafiscal, data_nota, id_produto, quantidade_produtos, preco_unit, total_item, data_criacao, data_atlz, versao))
    
    id_cliente = random.choice(lista_clientes)
    id_loja = random.choice(lista_lojas)
    id_forma_pgmt = random.choice(lista_forma_pagamento)
    id_cfop = random.choice(lista_id_cfop)
    while True:
        chaveacesso = ''.join([str(random.randint(0, 9)) for _ in range(44)])
        if chaveacesso not in lista_chave_acesso_db:
            break
    lista_ids_nfs.append((id_notafiscal, id_cliente, id_loja, id_forma_pgmt, id_cfop, chaveacesso))
    qtde_nf_no_bancodados = qtde_nf_no_bancodados + 1

df_itemnota = pd.DataFrame(lista_notas_fiscais, columns=["ID_NOTAFISCAL", "DATA","ID_PRODUTO","QUANTIDADE","PRECO_UNIT",
                                                            "TOTAL_ITEM", "DATA_CRIACAO","DATA_ATLZ","VERSAO"])
df_ids_nfs = pd.DataFrame(lista_ids_nfs, columns=["ID_NOTAFISCAL", "ID_CLIENTE","ID_LOJA","ID_FORMA_PGMT","ID_CFOP",
                                                  "CHAVE_ACESSO"])

df_agrupado = df_itemnota.groupby("ID_NOTAFISCAL").agg({
                                                            "DATA": "first",  
                                                            
                                                            "DATA_CRIACAO": "first",  
                                                            "DATA_ATLZ": "first",  
                                                            "VERSAO": "first"
}).reset_index()
df_total_nota = round((df_itemnota.groupby("ID_NOTAFISCAL")["TOTAL_ITEM"].sum().reset_index()), 2)
df_total_nota.columns = ["ID_NOTAFISCAL", "TOTAL_NOTA"]
df_datas_notas = df_itemnota.groupby("ID_NOTAFISCAL")["DATA"].first().reset_index()
df_datas_notas.columns = ["ID_NOTAFISCAL", "DATA"]

df_nota_fiscal = pd.merge(df_total_nota, df_agrupado, on="ID_NOTAFISCAL", how="left")
df_nota_fiscal = pd.merge(df_nota_fiscal, df_ids_nfs, on="ID_NOTAFISCAL", how="left")

df_itemnota = pd.merge(df_itemnota, df_ids_nfs, on="ID_NOTAFISCAL", how="left")
df_itemnota = pd.merge(df_itemnota, df_total_nota, on="ID_NOTAFISCAL", how="left")

df_ITEMNOTAf = pd.DataFrame({
                'ID_NOTAFISCAL':  df_itemnota['ID_NOTAFISCAL'],
                'ID_PRODUTO':     df_itemnota['ID_PRODUTO'],
                'QUANTIDADE':     df_itemnota['QUANTIDADE'],
                'PRECO_UNIT':     df_itemnota['PRECO_UNIT'],
                'TOTAL_ITEM':     df_itemnota['TOTAL_ITEM'],
                'DATA_CRIACAO':   df_itemnota['DATA_CRIACAO'],
                'DATA_ATLZ':      df_itemnota['DATA_ATLZ'],
                'VERSAO':         df_itemnota['VERSAO'],
})

df_NOTAFISCALf = pd.DataFrame({
                'DATA':           df_nota_fiscal['DATA'],
                'NUMERONF':       df_nota_fiscal['ID_NOTAFISCAL'],
                'TOTAL_NOTA':     df_nota_fiscal['TOTAL_NOTA'],
                'ID_FORMA_PGMT':  df_nota_fiscal['ID_FORMA_PGMT'],
                'ID_CLIENTE':     df_nota_fiscal['ID_CLIENTE'],
                'ID_LOJA':        df_nota_fiscal['ID_LOJA'],
                'ID_CFOP':        df_nota_fiscal['ID_CFOP'],
                'CHAVE_ACESSO':   df_ids_nfs['CHAVE_ACESSO'],
                'DATA_CRIACAO':   df_nota_fiscal['DATA_CRIACAO'],
                'DATA_ATLZ':      df_nota_fiscal['DATA_ATLZ'],
                'VERSAO':         df_nota_fiscal['VERSAO'],
})

df_VENDAf = pd.DataFrame({
                'DATA':           df_itemnota['DATA'],
                'ID_NOTAFISCAL':  df_itemnota['ID_NOTAFISCAL'],
                'ID_CLIENTE':     df_itemnota['ID_CLIENTE'],
                'ID_LOJA':        df_itemnota['ID_LOJA'],
                'ID_FORMA_PGMT':  df_itemnota['ID_FORMA_PGMT'],
                'TOTAL_VENDA':    df_itemnota['TOTAL_NOTA'],
                'DATA_CRIACAO':   df_itemnota['DATA_CRIACAO'],
                'DATA_ATLZ':      df_itemnota['DATA_ATLZ'],
                'VERSAO':         df_itemnota['VERSAO'],
})

display(df_ITEMNOTAf)
display(df_NOTAFISCALf)
display(df_VENDAf)


Conexão com o banco de dados estabelecida com sucesso
Query executada com sucesso
A conexão foi bem-sucedida


,IDPRODUTO,PRECO_CUSTO,PRECO_UNIT
0,1,6.70,8.54
1,2,4.19,6.30
2,3,5.12,7.49
3,4,2.96,3.62
4,5,20.79,30.55
...,...,...,...
98054,98055,58.05,88.19
98055,98056,8.47,11.75
98056,98057,9.96,14.58
98057,98058,2.59,3.65


,ID_NOTAFISCAL,ID_PRODUTO,QUANTIDADE,PRECO_UNIT,TOTAL_ITEM,DATA_CRIACAO,DATA_ATLZ,VERSAO
0,2801,97160,3,18.14,54.42,2025-03-19,None,1
1,2802,82225,1,2.73,2.73,2025-03-19,None,1
2,2803,33114,1,12.26,12.26,2025-03-19,None,1
3,2803,24536,5,83.13,415.65,2025-03-19,None,1
4,2803,13468,7,12.76,89.32,2025-03-19,None,1
5,2803,48247,7,69.35,485.45,2025-03-19,None,1
6,2803,40341,4,15.90,63.60,2025-03-19,None,1
7,2803,81683,3,20.30,60.90,2025-03-19,None,1
8,2803,65121,10,35.12,351.20,2025-03-19,None,1
9,2803,29698,1,8.01,8.01,2025-03-19,None,1


,DATA,NUMERONF,TOTAL_NOTA,ID_FORMA_PGMT,ID_CLIENTE,ID_LOJA,ID_CFOP,CHAVE_ACESSO,DATA_CRIACAO,DATA_ATLZ,VERSAO
0,2024-01-06 10:47:07,2801,54.42,1,357695,77,4,84068482887489160577387279961821445082947068,2025-03-19,None,1
1,2024-01-12 08:15:52,2802,2.73,5,787807,37,5,74912674700779663165985589330628066713684869,2025-03-19,None,1
2,2024-01-12 19:36:22,2803,2112.99,1,536336,46,3,88974381852035802322425902452099132251350737,2025-03-19,None,1


,DATA,ID_NOTAFISCAL,ID_CLIENTE,ID_LOJA,ID_FORMA_PGMT,TOTAL_VENDA,DATA_CRIACAO,DATA_ATLZ,VERSAO
0,2024-01-06 10:47:07,2801,357695,77,1,54.42,2025-03-19,None,1
1,2024-01-12 08:15:52,2802,787807,37,5,2.73,2025-03-19,None,1
2,2024-01-12 19:36:22,2803,536336,46,1,2112.99,2025-03-19,None,1
3,2024-01-12 19:36:22,2803,536336,46,1,2112.99,2025-03-19,None,1
4,2024-01-12 19:36:22,2803,536336,46,1,2112.99,2025-03-19,None,1
5,2024-01-12 19:36:22,2803,536336,46,1,2112.99,2025-03-19,None,1
6,2024-01-12 19:36:22,2803,536336,46,1,2112.99,2025-03-19,None,1
7,2024-01-12 19:36:22,2803,536336,46,1,2112.99,2025-03-19,None,1
8,2024-01-12 19:36:22,2803,536336,46,1,2112.99,2025-03-19,None,1
9,2024-01-12 19:36:22,2803,536336,46,1,2112.99,2025-03-19,None,1
